
# Diseño de software para cómputo científico

----

## Unidad 6: Interfaces de línea de comandos


## Referencias

- https://docs.python.org/3/library/argparse.html
- https://es.slideshare.net/tisto/argparse-python-command-line-parser
- https://en.wikipedia.org/wiki/Command-line_interface


## Estandar posix

```bash 
$ ejecutable subcomando -or --opcion -k foo
```

- En sistemas tipo Unix, el guión comienza las opciones-
- La convencion **GNU** es usar dos guiones y luego una palabra (por ejemplo, `--crear`) para identificar el uso de la opción
- Mientras que la convención anterior (y aún disponible como una opción para las opciones de uso frecuente) es usar *un guión** y luego uno letra (por ejemplo, `-c`).
- Si un guión va seguido de dos o más letras, puede significar que se están especificando dos opciones (`-or`), o puede significar que la segunda letra y las siguientes son un parámetro (como nombre de archivo o fecha) para la primera opción.
- Un doble-guión se usan para "opciones largas" donde se usan nombres de opciones más descriptivos. 
- Esta es una característica común del software GNU.

## `argparse`: analizador de opciones de línea de comandos, argumentos y subcomandos

```bash
python foo.py -a 1 --foo
```

- Los parámetros de una comando de python estan en `sys.argv`

    ```python
    >>> import sys
    >>> sys.argv
    ["foo.py", "-a", "1", "--foo"]
    ```
- El módulo `argparse` facilita la escritura de interfaces de línea de comandos. 
- El programa define qué argumentos requiere, y argparse descubrirá cómo analizarlos desde `sys.argv`. 
- El módulo argparse también genera automáticamente mensajes de ayuda y emite errores cuando los usuarios le dan argumentos inválidos al programa.
    
    

## `argparse`: analizador de opciones de línea de comandos, argumentos y subcomandos

```python
# prog.py
import argparse

parser = argparse.ArgumentParser(description='Process some integers.')
parser.add_argument('integers', metavar='N', type=int, nargs='+',
                    help='an integer for the accumulator')
parser.add_argument('--sum', dest='accumulate', action='store_const',
                    const=sum, default=max,
                    help='sum the integers (default: find the max)')

args = parser.parse_args()
print(args.accumulate(args.integers))
```

## `argparse`

```bash
$ python prog.py -h
usage: prog.py [-h] [--sum] N [N ...]

Process some integers.

positional arguments:
 N           an integer for the accumulator

optional arguments:
 -h, --help  show this help message and exit
 --sum       sum the integers (default: find the max)
    ```

## `argparse`

```bash
$ python prog.py 1 2 3 4
4

$ python prog.py 1 2 3 4 --sum
10
```

## `argparse`

```bash
$ python prog.py a b c
usage: prog.py [-h] [--sum] N [N ...]
prog.py: error: argument N: invalid int value: 'a'
```

## Clase `argparse.ArgumentParser`

Soporta los entre otros los siguientes parámetros los cuales deben pasarse como keywords:

- `prog` - El nombre del programa (default: sys.argv[0])
- `usage` - Como usar el programa (default: generated from arguments added to parser)
- `description` - Texto previo a la descripción de las ayudas (default: none)
- `epilog` - Texto posterior a las ayudas (default: none)


## `ArgumentParser.add_argument()`


- `name` or `flags` - Ya sea un nombre o una lista de cadenas de opciones(`-f`, `--foo`).
- `action` - Qué hacer con el parámetro (siguiente slide).
- `nargs` - Número de argumentos que tienen que ser consumidos por el parámetro.
- `const` - Una constante (aveces esto es requerido)
- `default` - Valor por defecto si no es provisto ninguno.
- `type` - Tipo de dato.
- `choices` - Valores aceptados por el argumento.
- `required` - Si es o no requerido (por defecto es `False`)
- `help` - Una breve descripción de lo que hace el argumento.
- `metavar` - Un nombre para el argumento en los mensajes de uso.
- `dest` - El nombre del atributo que se agregará al objeto devuelto por `parse_args()`.

### `ArgumentParser.add_argument()` -- name or flags

In [1]:
import argparse

parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('-f', '--foo')
parser.add_argument('bar')
namespace = parser.parse_args(['BAR'])
namespace

Namespace(bar='BAR', foo=None)

### `ArgumentParser.add_argument()` -- action 

#### `store`

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', action="store")
parser.parse_args(['--foo', '1'])

Namespace(foo='1')

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', action='store_const', const=42)
parser.parse_args(['--foo'])

Namespace(foo=42)

### `ArgumentParser.add_argument()` -- action 

#### `store_true` and `store_false`

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', action='store_true')
parser.add_argument('--bar', action='store_false')
parser.add_argument('--baz', action='store_false')
parser.parse_args(['--foo', '--bar'])

Namespace(bar=False, baz=True, foo=True)

####  `append`

In [5]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', action='append')
parser.parse_args('--foo 1 --foo 2'.split())

Namespace(foo=['1', '2'])

### `ArgumentParser.add_argument()` -- action 

#### `append_const`

In [6]:
parser = argparse.ArgumentParser()
parser.add_argument('--str', dest='types', action='append_const', const=str)
parser.add_argument('--int', dest='types', action='append_const', const=int)
parser.parse_args('--str --int'.split())

Namespace(types=[<class 'str'>, <class 'int'>])

####  `count`

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument('--verbose', '-v', action='count')
parser.parse_args(['-vvv'])

Namespace(verbose=3)

#### `version`

In [8]:
import argparse
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('--version', action='version', version='%(prog)s 2.0')
parser.parse_args(['--version'])

PROG 2.0


SystemExit: 0

/home/juan/proyectos/diseno_sci_sfw/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### `ArgumentParser.add_argument()` -- type

- Por defecto, los objetos ArgumentParser leen los argumentos de la línea de comandos como `str`. 
- Sin embargo, con bastante frecuencia, la cadena de línea de comandos debe interpretarse como otro tipo, como float o int.

In [9]:
parser = argparse.ArgumentParser()
parser.add_argument('foo', type=int)
parser.add_argument('bar', type=float)
parser.parse_args('2 35.58'.split())

Namespace(bar=35.58, foo=2)

Para facilitar el uso de varios tipos de archivos, el módulo argparse proporciona el FileType

In [10]:
parser = argparse.ArgumentParser()
parser.add_argument('bar', type=argparse.FileType('w'))
parser.parse_args(['out.txt'])

Namespace(bar=<_io.TextIOWrapper name='out.txt' mode='w' encoding='UTF-8'>)

Cualquier `callable` puede utilizarse como *type*

### `ArgumentParser.add_argument()` -- choices

Algunos argumentos de la línea de comandos deben seleccionarse de un conjunto restringido de valores. 

In [11]:
parser = argparse.ArgumentParser(prog='game.py')
parser.add_argument('move', choices=['rock', 'paper', 'scissors'])
parser.parse_args(['rock'])

Namespace(move='rock')

In [12]:
parser.parse_args(['fire'])

usage: game.py [-h] {rock,paper,scissors}
game.py: error: argument move: invalid choice: 'fire' (choose from 'rock', 'paper', 'scissors')


SystemExit: 2

## Subcomandos

- Muchos programas dividen su funcionalidad en varios subcomandos, por ejemplo, el programa **git** puede invocar subcomandos como `git push`, `git commit` o `git pull`
- Dividir la funcionalidad de esta manera puede ser una idea particularmente buena cuando un programa realiza varias funciones diferentes que requieren diferentes tipos de argumentos de línea de comandos.

In [13]:
# create the top-level parser
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('--foo', action='store_true', help='foo help')
subparsers = parser.add_subparsers(help='sub-command help')

# create the parser for the "a" command
parser_a = subparsers.add_parser('a', help='a help')
parser_a.add_argument('bar', type=int, help='bar help')

# create the parser for the "b" command
parser_b = subparsers.add_parser('b', help='b help')
parser_b.add_argument('--baz', choices='XYZ', help='baz help');

In [14]:
parser.parse_args(['a', '12'])

Namespace(bar=12, foo=False)

In [15]:
parser.parse_args(['--foo', 'b', '--baz', 'Z'])

Namespace(baz='Z', foo=True)

## Subcomandos

Una forma particularmente efectiva de manejar subcomandos es combinar el uso del método `add_subparsers()` con llamadas a `set_defaults()` para que cada subparser sepa qué función de Python debe ejecutar. Por ejemplo:

In [20]:
# sub-command functions
def foo(x, y):
    print(x * y)

def bar(z):
    print('((%s))' % args.z)

# create the top-level parser
parser = argparse.ArgumentParser()
subparsers = parser.add_subparsers()

# create the parser for the "foo" command
parser_foo = subparsers.add_parser('foo')
parser_foo.add_argument('-x', type=int, default=1)
parser_foo.add_argument('y', type=float)
parser_foo.set_defaults(handler=foo)

# create the parser for the "bar" command
parser_bar = subparsers.add_parser('bar')
parser_bar.add_argument('z')
parser_bar.set_defaults(handler=bar);

## Subcomandos

In [ ]:
ns = parser.parse_args('foo 1 -x 2'.split())
ns

In [22]:
# extraemos todos los argumentos como keywords
kwargs = {k: v for k, v in vars(ns).items() if k != "handler"}
kwargs

{'x': 2, 'y': 1.0}

In [23]:
ns.handler(**kwargs)

2.0


In [24]:
def parse_and_run(parser, argv):
    ns = parser.parse_args('foo 1 -x 2'.split())
    kwargs = {k: v for k, v in vars(ns).items() if k != "handler"}
    return ns.handler(**kwargs)

In [25]:
parse_and_run(parser, ["bar", "23"])

2.0
